## Import Data

In [ ]:
import tensorflow as tf
train,test = tf.keras.datasets.mnist.load_data(path="mnist.npz")

In [ ]:

import numpy as np

X_train = train[0]
X_train = X_train.reshape((60000,28,28,1))
import numpy as np
def to_one_hot(y, num_class=10):
  master = []
  for value in y:
    row = np.zeros(num_class)
    row[value] = 1
    master.append(row)
  return np.array(master)

y_train = to_one_hot(train[1], num_class =10)
X_train = X_train.astype("float32")

In [ ]:
X_train_flat = train[0].reshape((60000,784))
y_train = to_one_hot(train[1], num_class =10)
X_train_flat = X_train_flat.astype("float32")

In [ ]:
import numpy as np

X_test = test[0]
X_test = X_test.reshape((10000,28,28,1))
import numpy as np
def to_one_hot(y, num_class=10):
  master = []
  for value in y:
    row = np.zeros(num_class)
    row[value] = 1
    master.append(row)
  return np.array(master)

y_test = to_one_hot(test[1], num_class =10)
X_test = X_test.astype("float32")

In [ ]:
X_test_flat = test[0].reshape((10000,784))
y_test = to_one_hot(test[1], num_class =10)
X_test_flat = X_test_flat.astype("float32")

## Noisy Data

In [ ]:
noise_factor = 0.3
X_train_noisy = X_train + noise_factor * np.random.normal(loc=0.0, scale=255, size=X_train.shape)

In [ ]:
X_train_flat_noisy = X_train_noisy.reshape((60000,784))

# Making our Models

## Building ResNet

In [ ]:
from keras.layers import Input, Conv2D, ReLU, BatchNormalization,Add, AveragePooling2D, Flatten, Dense

#Resnet code inspired by
#https://towardsdatascience.com/understanding-and-coding-a-resnet-in-keras-446d7ff84d33

#relu based batch normalization
#given a tensor x, we apply relu on it and use Batch Normalization on it
def relu_bn(inputs):
    relu = ReLU()(inputs)
    bn = BatchNormalization()(relu)
    return bn

# a skip block with 2 convulotional layers
#size is the size of the layer
#Note that stride > 1 is downsampling. So we must downsample the original tensor as well to be able to add it to the downsampled one
def skip_block(x, filters, stride = 1, size = 3):
  y = Conv2D(kernel_size=size,
              strides= stride,
              filters=filters,
              padding="same")(x)
  y = relu_bn(y)
  y = Conv2D(kernel_size=size,
              strides=1,
              filters=filters,
              padding="same")(y)

  #downsample the original tensor
  #note that we could in theory just use max pooling or something here
  if stride != 1:
      x = Conv2D(kernel_size=1,
                  strides=stride,
                  filters=filters,
                  padding="same")(x)
  out = Add()([x, y])
  out = relu_bn(out)
  return out


from keras.models import Model
from keras import optimizers


#creates a Resnet model
#takes as input number of skip connections and when to downsample
#the default parameters are taken from the slides of CS 583
def ResNet(ns = [3,5,5,3], initial_filters = 64):
  inputs = Input(shape = (28,28,1))

  current_filters = initial_filters

  t = BatchNormalization()(inputs)

  t = Conv2D(kernel_size=3, strides=1, filters=current_filters, padding="same")(t)
  t = relu_bn(t)

  for cnt, block in enumerate(ns):
    #Downscale if starting a new block unless it is the first block
    if cnt != 0:
      current_filters *= 2
      t = skip_block(t, current_filters, 2, 3)
    else:
      t = skip_block(t, current_filters, 1, 3)
    #make skip connections
    for connection in range(block-1):
      t = skip_block(t, current_filters, 1, 3)
    
  #Average pool and flatten followed by a dense layer
  t = AveragePooling2D(4)(t)
  t = Flatten()(t)
  outputs = Dense(10, activation='softmax')(t)
  
  model = Model(inputs, outputs)

  return model

In [ ]:
modelRN = ResNet()
learning_rate = 1E-4
modelRN.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=learning_rate),
              metrics=['acc'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
import tensorflow as tf


In [ ]:


tf.keras.models.save_model(modelRN, "/content/drive/MyDrive/Trick_NN/Models/MNIST_Models/RN_arch.h5")

In [ ]:
modelRN.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 28, 28, 1)    4           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 28, 28, 64)   640         batch_normalization[0][0]        
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 28, 28, 64)   0           conv2d[0][0]                     
______________________________________________________________________________________________

## Basic 1-hidden layer neural network

In [ ]:
from keras.layers import Dense, Input
from keras import models

input_feat = Input(shape=(28,28,1))

f1 = Flatten()(input_feat)
hidden1 = Dense(50, activation='relu')(f1)
output = Dense(10, activation='softmax')(hidden1)

classifier = models.Model(input_feat, output)

classifier.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 50)                39250     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                510       
Total params: 39,760
Trainable params: 39,760
Non-trainable params: 0
_________________________________________________________________


In [ ]:
classifier.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.RMSprop(lr=1E-3),
                  metrics=['acc'])

tf.keras.models.save_model(classifier, "/content/drive/MyDrive/Trick_NN/Models/MNIST_Models/Dense_arch.h5")

# Training Models

In [ ]:
classifier.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.RMSprop(lr=1E-3),
                  metrics=['acc'])

history = classifier.fit(X_train, y_train, epochs=10, validation_split=0.2)

Epoch 1/10
1500/1500 [==============================] - 6s 3ms/step - loss: 5.8873 - acc: 0.7007 - val_loss: 0.7046 - val_acc: 0.8882
Epoch 2/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.6396 - acc: 0.8945 - val_loss: 0.5484 - val_acc: 0.9128
Epoch 3/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.4883 - acc: 0.9180 - val_loss: 0.5655 - val_acc: 0.9141
Epoch 4/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.4390 - acc: 0.9266 - val_loss: 0.5107 - val_acc: 0.9247
Epoch 5/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.4220 - acc: 0.9327 - val_loss: 0.5186 - val_acc: 0.9230
Epoch 6/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3890 - acc: 0.9354 - val_loss: 0.5776 - val_acc: 0.9234
Epoch 7/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3658 - acc: 0.9394 - val_loss: 0.6959 - val_acc: 0.9248
Epoch 8/10
1500/1500 [==============================] - 4s 3ms

In [ ]:
classifier.save_weights("/content/drive/MyDrive/Trick_NN/Models/MNIST_Models/Dense.h5")

In [ ]:
input_feat = Input(shape=(28,28,1))

f1 = Flatten()(input_feat)
hidden1 = Dense(50, activation='relu')(f1)
output = Dense(10, activation='softmax')(hidden1)

classifier = models.Model(input_feat, output)

classifier.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.RMSprop(lr=1E-3),
                  metrics=['acc'])

history = classifier.fit(X_train_noisy, y_train, epochs=15, validation_split=0.2)

Epoch 1/15
1500/1500 [==============================] - 6s 2ms/step - loss: 12.7693 - acc: 0.4209 - val_loss: 1.4337 - val_acc: 0.6593
Epoch 2/15
1500/1500 [==============================] - 3s 2ms/step - loss: 1.1651 - acc: 0.6981 - val_loss: 1.4144 - val_acc: 0.7445
Epoch 3/15
1500/1500 [==============================] - 3s 2ms/step - loss: 0.9303 - acc: 0.7706 - val_loss: 1.4756 - val_acc: 0.7798
Epoch 4/15
1500/1500 [==============================] - 3s 2ms/step - loss: 0.8143 - acc: 0.8082 - val_loss: 1.5035 - val_acc: 0.8054
Epoch 5/15
1500/1500 [==============================] - 3s 2ms/step - loss: 0.7478 - acc: 0.8281 - val_loss: 1.7663 - val_acc: 0.8226
Epoch 6/15
1500/1500 [==============================] - 3s 2ms/step - loss: 0.6749 - acc: 0.8475 - val_loss: 1.7148 - val_acc: 0.8113
Epoch 7/15
1500/1500 [==============================] - 3s 2ms/step - loss: 0.6401 - acc: 0.8578 - val_loss: 2.0669 - val_acc: 0.8332
Epoch 8/15
1500/1500 [==============================] - 3s 2m

In [ ]:
classifier.save_weights("/content/drive/MyDrive/Trick_NN/Models/MNIST_Models/DenseNoisy3.h5")

In [ ]:
modelRN.fit(X_train,y_train, epochs=3, validation_split=0.2)

Epoch 1/3
1500/1500 [==============================] - 97s 50ms/step - loss: 0.4993 - acc: 0.8347 - val_loss: 0.0577 - val_acc: 0.9839
Epoch 2/3
1500/1500 [==============================] - 74s 50ms/step - loss: 0.0630 - acc: 0.9818 - val_loss: 0.0863 - val_acc: 0.9772
Epoch 3/3
1500/1500 [==============================] - 74s 50ms/step - loss: 0.0407 - acc: 0.9876 - val_loss: 0.0415 - val_acc: 0.9897


In [ ]:
modelRN.save_weights("/content/drive/MyDrive/Trick_NN/Models/MNIST_Models/RNN.h5")

In [ ]:
modelRN = ResNet()
learning_rate = 1E-4
modelRN.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=learning_rate),
              metrics=['acc'])

modelRN.fit(X_train_noisy,y_train, epochs=2, validation_split=0.2)

Epoch 1/2
1500/1500 [==============================] - 54s 32ms/step - loss: 0.7645 - acc: 0.7398 - val_loss: 0.1276 - val_acc: 0.9617
Epoch 2/2
1500/1500 [==============================] - 48s 32ms/step - loss: 0.0833 - acc: 0.9759 - val_loss: 0.0874 - val_acc: 0.9763


In [ ]:
modelRN.save_weights("/content/drive/MyDrive/Trick_NN/Models/MNIST_Models/RNN_Noisy3.h5")

Evaluation on test set

In [ ]:
classifier.load_weights("/content/drive/MyDrive/Trick_NN/Models/MNIST_Models/Dense.h5")
classifier.evaluate(X_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.7812 - acc: 0.9300


[0.7811885476112366, 0.9300000071525574]

In [ ]:
classifier.load_weights("/content/drive/MyDrive/Trick_NN/Models/MNIST_Models/DenseNoisy.h5")
classifier.evaluate(X_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.6429 - acc: 0.9279


[0.6428645849227905, 0.9279000163078308]

In [ ]:
modelRN.load_weights("/content/drive/MyDrive/Trick_NN/Models/MNIST_Models/RNN.h5")
modelRN.evaluate(X_test, y_test)

313/313 [==============================] - 3s 9ms/step - loss: 0.0342 - acc: 0.9901


[0.034244924783706665, 0.9901000261306763]

In [ ]:
modelRN.load_weights("/content/drive/MyDrive/Trick_NN/Models/MNIST_Models/RNN_Noisy3.h5")
modelRN.evaluate(X_test, y_test)

313/313 [==============================] - 36s 9ms/step - loss: 0.0909 - acc: 0.9747


[0.07397224009037018, 0.9793000221252441]